# Data Collection:

# Web scraping all reviews for professors in profs dataframe

## Setup, imports

In [41]:
# import modules
import pandas as pd
import requests
import requests_cache
import json
import time
from tqdm import tqdm, trange

In [2]:
# set up cache
#requests_cache.install_cache("reviews")

## Request data from RMP

In [3]:
# endpoint and header for POST request
url = 'https://www.ratemyprofessors.com/graphql'
headers = {
    "Authorization": "Basic dGVzdDp0ZXN0", 
}

### Experimenting: get reviews for first professor (Robert Borgen)

In [66]:
data = {
    "query": "query RatingsListQuery(\n  $count: Int!\n  $id: ID!\n  $courseFilter: String\n  $cursor: String\n) {\n  node(id: $id) {\n    __typename\n    ... on Teacher {\n      ...RatingsList_teacher_4pguUW\n    }\n    id\n  }\n}\n\nfragment RatingsList_teacher_4pguUW on Teacher {\n  id\n  legacyId\n  lastName\n  numRatings\n  school {\n    id\n    legacyId\n    name\n    city\n    state\n    avgRating\n    numRatings\n  }\n  ...Rating_teacher\n  ...NoRatingsArea_teacher\n  ratings(first: $count, after: $cursor, courseFilter: $courseFilter) {\n    edges {\n      cursor\n      node {\n        ...Rating_rating\n        id\n        __typename\n      }\n    }\n    pageInfo {\n      hasNextPage\n      endCursor\n    }\n  }\n}\n\nfragment Rating_teacher on Teacher {\n  ...RatingFooter_teacher\n  ...RatingSuperHeader_teacher\n  ...ProfessorNoteSection_teacher\n}\n\nfragment NoRatingsArea_teacher on Teacher {\n  lastName\n  ...RateTeacherLink_teacher\n}\n\nfragment Rating_rating on Rating {\n  comment\n  flagStatus\n  createdByUser\n  teacherNote {\n    id\n  }\n  ...RatingHeader_rating\n  ...RatingSuperHeader_rating\n  ...RatingValues_rating\n  ...CourseMeta_rating\n  ...RatingTags_rating\n  ...RatingFooter_rating\n  ...ProfessorNoteSection_rating\n}\n\nfragment RatingHeader_rating on Rating {\n  date\n  class\n  helpfulRating\n  clarityRating\n  isForOnlineClass\n}\n\nfragment RatingSuperHeader_rating on Rating {\n  legacyId\n}\n\nfragment RatingValues_rating on Rating {\n  helpfulRating\n  clarityRating\n  difficultyRating\n}\n\nfragment CourseMeta_rating on Rating {\n  attendanceMandatory\n  wouldTakeAgain\n  grade\n  textbookUse\n  isForOnlineClass\n  isForCredit\n}\n\nfragment RatingTags_rating on Rating {\n  ratingTags\n}\n\nfragment RatingFooter_rating on Rating {\n  id\n  comment\n  adminReviewedAt\n  flagStatus\n  legacyId\n  thumbsUpTotal\n  thumbsDownTotal\n  thumbs {\n    userId\n    thumbsUp\n    thumbsDown\n    id\n  }\n  teacherNote {\n    id\n  }\n}\n\nfragment ProfessorNoteSection_rating on Rating {\n  teacherNote {\n    ...ProfessorNote_note\n    id\n  }\n  ...ProfessorNoteEditor_rating\n}\n\nfragment ProfessorNote_note on TeacherNotes {\n  comment\n  ...ProfessorNoteHeader_note\n  ...ProfessorNoteFooter_note\n}\n\nfragment ProfessorNoteEditor_rating on Rating {\n  id\n  legacyId\n  class\n  teacherNote {\n    id\n    teacherId\n    comment\n  }\n}\n\nfragment ProfessorNoteHeader_note on TeacherNotes {\n  createdAt\n  updatedAt\n}\n\nfragment ProfessorNoteFooter_note on TeacherNotes {\n  legacyId\n  flagStatus\n}\n\nfragment RateTeacherLink_teacher on Teacher {\n  legacyId\n  numRatings\n  lockStatus\n}\n\nfragment RatingFooter_teacher on Teacher {\n  id\n  legacyId\n  lockStatus\n  isProfCurrentUser\n}\n\nfragment RatingSuperHeader_teacher on Teacher {\n  firstName\n  lastName\n  legacyId\n  school {\n    name\n    id\n  }\n}\n\nfragment ProfessorNoteSection_teacher on Teacher {\n  ...ProfessorNote_teacher\n  ...ProfessorNoteEditor_teacher\n}\n\nfragment ProfessorNote_teacher on Teacher {\n  ...ProfessorNoteHeader_teacher\n  ...ProfessorNoteFooter_teacher\n}\n\nfragment ProfessorNoteEditor_teacher on Teacher {\n  id\n}\n\nfragment ProfessorNoteHeader_teacher on Teacher {\n  lastName\n}\n\nfragment ProfessorNoteFooter_teacher on Teacher {\n  legacyId\n  isProfCurrentUser\n}\n",
    "variables":{
        "count": 50,
        "id": "VGVhY2hlci05NTY2",
        #"courseFilter": null,
        #"cursor": "YXJyYXljb25uZWN0aW9uOjE5"
    }
}

In [67]:
# response/check status
response = requests.post(url, headers = headers, json=data)
response.raise_for_status()

In [95]:
profInfoDict = {i:response.json()['data']['node'][i] for i in response.json()['data']['node'] if i in ['firstName', 'lastName', 'id', 'numRatings']}
profInfoDict

{'firstName': 'Robert',
 'id': 'VGVhY2hlci05NTY2',
 'lastName': 'Borgen',
 'numRatings': 39}

In [100]:
profInfo = pd.DataFrame(profInfoDict, index=[0])
profInfo

,firstName,id,lastName,numRatings
0,Robert,VGVhY2hlci05NTY2,Borgen,39


In [114]:
response.json()

{'data': {'node': {'__typename': 'Teacher',
   'firstName': 'Robert',
   'id': 'VGVhY2hlci05NTY2',
   'isProfCurrentUser': False,
   'lastName': 'Borgen',
   'legacyId': 9566,
   'lockStatus': 'none',
   'numRatings': 39,
   'ratings': {'edges': [{'cursor': 'YXJyYXljb25uZWN0aW9uOjA=',
      'node': {'__typename': 'Rating',
       'adminReviewedAt': '2011-01-26 17:30:48 +0000 UTC',
       'attendanceMandatory': '',
       'clarityRating': 5,
       'class': 'HIS194',
       'comment': 'He is amazing! I had lots of fun in his class. You just have to get used to it because it may seem a little bit odd.',
       'createdByUser': False,
       'date': '2011-01-26 17:21:05 +0000 UTC',
       'difficultyRating': 1,
       'flagStatus': 'UNFLAGGED',
       'grade': '',
       'helpfulRating': 5,
       'id': 'UmF0aW5nLTE4MTg0MzY4',
       'isForCredit': False,
       'isForOnlineClass': False,
       'legacyId': 18184368,
       'ratingTags': '',
       'teacherNote': None,
       'textbookUse

In [77]:
response.json()['data']['node']

{'__typename': 'Teacher',
 'firstName': 'Robert',
 'id': 'VGVhY2hlci05NTY2',
 'isProfCurrentUser': False,
 'lastName': 'Borgen',
 'legacyId': 9566,
 'lockStatus': 'none',
 'numRatings': 39,
 'ratings': {'edges': [{'cursor': 'YXJyYXljb25uZWN0aW9uOjA=',
    'node': {'__typename': 'Rating',
     'adminReviewedAt': '2011-01-26 17:30:48 +0000 UTC',
     'attendanceMandatory': '',
     'clarityRating': 5,
     'class': 'HIS194',
     'comment': 'He is amazing! I had lots of fun in his class. You just have to get used to it because it may seem a little bit odd.',
     'createdByUser': False,
     'date': '2011-01-26 17:21:05 +0000 UTC',
     'difficultyRating': 1,
     'flagStatus': 'UNFLAGGED',
     'grade': '',
     'helpfulRating': 5,
     'id': 'UmF0aW5nLTE4MTg0MzY4',
     'isForCredit': False,
     'isForOnlineClass': False,
     'legacyId': 18184368,
     'ratingTags': '',
     'teacherNote': None,
     'textbookUse': 5,
     'thumbs': [],
     'thumbsDownTotal': 0,
     'thumbsUpTotal'

In [68]:
response.json()['data']['node']['ratings']['edges']

[{'cursor': 'YXJyYXljb25uZWN0aW9uOjA=',
  'node': {'__typename': 'Rating',
   'adminReviewedAt': '2011-01-26 17:30:48 +0000 UTC',
   'attendanceMandatory': '',
   'clarityRating': 5,
   'class': 'HIS194',
   'comment': 'He is amazing! I had lots of fun in his class. You just have to get used to it because it may seem a little bit odd.',
   'createdByUser': False,
   'date': '2011-01-26 17:21:05 +0000 UTC',
   'difficultyRating': 1,
   'flagStatus': 'UNFLAGGED',
   'grade': '',
   'helpfulRating': 5,
   'id': 'UmF0aW5nLTE4MTg0MzY4',
   'isForCredit': False,
   'isForOnlineClass': False,
   'legacyId': 18184368,
   'ratingTags': '',
   'teacherNote': None,
   'textbookUse': 5,
   'thumbs': [],
   'thumbsDownTotal': 0,
   'thumbsUpTotal': 0,
   'wouldTakeAgain': None}},
 {'cursor': 'YXJyYXljb25uZWN0aW9uOjE=',
  'node': {'__typename': 'Rating',
   'adminReviewedAt': '2010-04-06 11:26:35 +0000 UTC',
   'attendanceMandatory': '',
   'clarityRating': 2,
   'class': 'HIS194A',
   'comment': 'B

In [69]:
# put response info into a data frame
cursorDF = pd.DataFrame(response.json()['data']['node']['ratings']['edges'])
cursorDF

,cursor,node
0,YXJyYXljb25uZWN0aW9uOjA=,"{'__typename': 'Rating', 'adminReviewedAt': '2..."
1,YXJyYXljb25uZWN0aW9uOjE=,"{'__typename': 'Rating', 'adminReviewedAt': '2..."
2,YXJyYXljb25uZWN0aW9uOjI=,"{'__typename': 'Rating', 'adminReviewedAt': '2..."
3,YXJyYXljb25uZWN0aW9uOjM=,"{'__typename': 'Rating', 'adminReviewedAt': '2..."
4,YXJyYXljb25uZWN0aW9uOjQ=,"{'__typename': 'Rating', 'adminReviewedAt': '2..."
5,YXJyYXljb25uZWN0aW9uOjU=,"{'__typename': 'Rating', 'adminReviewedAt': '2..."
6,YXJyYXljb25uZWN0aW9uOjY=,"{'__typename': 'Rating', 'adminReviewedAt': '2..."
7,YXJyYXljb25uZWN0aW9uOjc=,"{'__typename': 'Rating', 'adminReviewedAt': '2..."
8,YXJyYXljb25uZWN0aW9uOjg=,"{'__typename': 'Rating', 'adminReviewedAt': '2..."
9,YXJyYXljb25uZWN0aW9uOjk=,"{'__typename': 'Rating', 'adminReviewedAt': No..."


In [54]:
# inspect first node
cursorDF['node'][0]

{'__typename': 'Rating',
 'adminReviewedAt': '2011-01-26 17:30:48 +0000 UTC',
 'attendanceMandatory': '',
 'clarityRating': 5,
 'class': 'HIS194',
 'comment': 'He is amazing! I had lots of fun in his class. You just have to get used to it because it may seem a little bit odd.',
 'createdByUser': False,
 'date': '2011-01-26 17:21:05 +0000 UTC',
 'difficultyRating': 1,
 'flagStatus': 'UNFLAGGED',
 'grade': '',
 'helpfulRating': 5,
 'id': 'UmF0aW5nLTE4MTg0MzY4',
 'isForCredit': False,
 'isForOnlineClass': False,
 'legacyId': 18184368,
 'ratingTags': '',
 'teacherNote': None,
 'textbookUse': 5,
 'thumbs': [],
 'thumbsDownTotal': 0,
 'thumbsUpTotal': 0,
 'wouldTakeAgain': None}

In [74]:
# initialize dataframe for reviews
reviews = pd.DataFrame(columns=cursorDF['node'][0].keys())
reviews

,__typename,adminReviewedAt,attendanceMandatory,clarityRating,class,comment,createdByUser,date,difficultyRating,flagStatus,...,isForCredit,isForOnlineClass,legacyId,ratingTags,teacherNote,textbookUse,thumbs,thumbsDownTotal,thumbsUpTotal,wouldTakeAgain


In [75]:
# loop to populate reviews df
for i in trange(0, len(cursorDF)):
    
    # temporary dictionary
    tempDict = cursorDF['node'][i]
    
    # convert dict to temporary dataframe
    tempDF = pd.DataFrame([tempDict], index=[i])

    # append tempDF to reviews df
    reviews.loc[len(reviews)] = tempDF.iloc[0,:]

  0%|          | 0/39 [00:00<?, ?it/s]C:\Users\riley\AppData\Local\Temp\ipykernel_23656\2749055342.py:11: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  reviews.loc[len(reviews)] = tempDF.iloc[0,:]
C:\Users\riley\AppData\Local\Temp\ipykernel_23656\2749055342.py:11: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  reviews.loc[len(reviews)] = tempDF.iloc[0,:]
C:\Users\riley\AppData\Local\Temp\ipykernel_23656\2749055342.py:11: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  reviews.loc[len(reviews)] = tempDF.iloc[0,:]
C:\Users\riley\AppData\Local\Temp\ipykernel_23656\2749055342.py:11: FutureWarning: In a future ver

In [76]:
# view populated reviews df
reviews

,__typename,adminReviewedAt,attendanceMandatory,clarityRating,class,comment,createdByUser,date,difficultyRating,flagStatus,...,isForCredit,isForOnlineClass,legacyId,ratingTags,teacherNote,textbookUse,thumbs,thumbsDownTotal,thumbsUpTotal,wouldTakeAgain
0,Rating,2011-01-26 17:30:48 +0000 UTC,,5,HIS194,He is amazing! I had lots of fun in his class....,False,2011-01-26 17:21:05 +0000 UTC,1,UNFLAGGED,...,False,False,18184368,,None,5,[],0,0,None
1,Rating,2010-04-06 11:26:35 +0000 UTC,,2,HIS194A,"Bad prof, simply put. Thankfully, Borgen is ap...",False,2010-01-11 21:59:48 +0000 UTC,2,UNFLAGGED,...,False,False,16712359,,None,5,[],0,0,None
2,Rating,2010-04-01 21:59:55 +0000 UTC,,2,HIS194A,"Pretentious, disorganized, rants, and goes off...",False,2009-12-22 00:52:25 +0000 UTC,2,UNFLAGGED,...,False,False,16620461,,None,5,[],1,0,None
3,Rating,2010-04-01 00:55:36 +0000 UTC,,2,HIS194A,"Very disorganized in lecture, does not go into...",False,2009-12-20 04:13:29 +0000 UTC,3,UNFLAGGED,...,False,False,16608094,,None,3,[],0,0,None
4,Rating,2010-03-09 21:34:28 +0000 UTC,,3,JPN101,"Pretentious, scatter-brained, and enjoys belit...",False,2009-11-12 23:29:20 +0000 UTC,2,UNFLAGGED,...,False,False,16361756,,None,4,[],0,0,None
5,Rating,2008-07-13 19:39:08 +0000 UTC,,4,HIS194A,"A little disorganized at times, but the man kn...",False,2008-04-10 22:42:52 +0000 UTC,4,UNFLAGGED,...,False,False,14274644,,None,5,[],0,0,None
6,Rating,2008-07-13 17:35:49 +0000 UTC,,3,HIS194A,The class would have been easy if I had more t...,False,2008-04-03 00:37:37 +0000 UTC,5,UNFLAGGED,...,False,False,14235767,,None,5,[],0,0,None
7,Rating,2008-07-13 16:47:52 +0000 UTC,,4,HIS194A,He obviously enjoys the material quite a lot a...,False,2008-03-30 04:08:51 +0000 UTC,4,UNFLAGGED,...,False,False,14216866,,None,5,[],0,0,None
8,Rating,2008-07-13 11:28:08 +0000 UTC,,3,HIS194A,He is a good guy but there is ridiculous amoun...,False,2008-03-20 06:56:18 +0000 UTC,5,UNFLAGGED,...,False,False,14183721,,None,5,[],0,0,None
9,Rating,None,,4,HIS9B,Take good notes during his lecture and you'll ...,False,2007-03-29 19:24:06 +0000 UTC,3,REVIEWED,...,False,False,11218257,,None,None,[],0,0,None


In [102]:
# view columns
reviews.columns

Index(['__typename', 'adminReviewedAt', 'attendanceMandatory', 'clarityRating',
       'class', 'comment', 'createdByUser', 'date', 'difficultyRating',
       'flagStatus', 'grade', 'helpfulRating', 'id', 'isForCredit',
       'isForOnlineClass', 'legacyId', 'ratingTags', 'teacherNote',
       'textbookUse', 'thumbs', 'thumbsDownTotal', 'thumbsUpTotal',
       'wouldTakeAgain'],
      dtype='object')

In [113]:
reviews['thumbsDownTotal']

0     0
1     0
2     1
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
Name: thumbsDownTotal, dtype: object

In [106]:
# tidy columns
reviews = reviews.rename(columns={'id': 'reviewID'})